In [ ]:
# integrate this into 2_network_creation

import os
import sys

module = os.path.abspath(os.path.join('..'))
lib = os.path.abspath(os.path.join(module, os.pardir))
if lib not in sys.path:
    sys.path.append(lib)

from lib.experiments.utils.metrics import calc_shortest_paths
from lib.experiments.utils.data_repo_api import DataRepoAPI
from datetime import datetime as dt

# multiprocessing
from joblib import Parallel, delayed
from multiprocessing import cpu_count

# Experiment result output dir
output_dir = 'experiment_results'
if not output_dir in os.listdir():
    os.mkdir(output_dir)

In [ ]:
def run_sp_calc(generator, file_name):
    g = generator.generate()
    sp = calc_shortest_paths(g)

    # save to file
    with open(os.path.join(output_dir, file_name + '.pkl'), 'wb') as f:
        f.write(sp)

    # put to repo
    DataRepoAPI.update_or_create(
            file_name=file_name + '.pkl',
            file_path=output_dir,
            repo_path='network-metrics'
        )


In [ ]:
mng_pre = None
mng_post = None

plc_pre = None
plc_post = None

distanced_pre = None
distanced_post = None

s_paths = []

s_paths.append(lambda: run_sp_calc(mng_pre.generate(), 'sp_mng_pre'))
s_paths.append(lambda: run_sp_calc(mng_post.generate(), 'sp_mng_post'))
s_paths.append(lambda: run_sp_calc(plc_pre.generate(), 'sp_plc_pre'))
s_paths.append(lambda: run_sp_calc(plc_post.generate(), 'sp_plc_post'))
s_paths.append(lambda: run_sp_calc(distanced_pre.generate(), 'sp_dist_pre'))
s_paths.append(lambda: run_sp_calc(distanced_post.generate(), 'sp_dist_post'))

In [ ]:
num_cores = min(len(s_paths), cpu_count()-1, 6)
print(f"Running {num_cores} jobs on {cpu_count()} available cores...")
print("Start time %s" % dt.now())
with Parallel(n_jobs=num_cores) as processes:
    procs = processes(delayed(e)() for e in s_paths)
print("End time %s" % dt.now())
